In [1]:
!pip install pillow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!unzip "/content/drive/MyDrive/TRIALDETECTION/FashionBodyRegressionDetection.v3-328-images-2024-02-21-1-43pm.voc.zip" -d "/content/drive/MyDrive/TRIALDETECTION/unzip"

Archive:  /content/drive/MyDrive/TRIALDETECTION/FashionBodyRegressionDetection.v3-328-images-2024-02-21-1-43pm.voc.zip
 extracting: /content/drive/MyDrive/TRIALDETECTION/unzip/README.dataset.txt  
 extracting: /content/drive/MyDrive/TRIALDETECTION/unzip/README.roboflow.txt  
   creating: /content/drive/MyDrive/TRIALDETECTION/unzip/train/
 extracting: /content/drive/MyDrive/TRIALDETECTION/unzip/train/01249489-01_jpg.rf.a59115a2ff0c036a5d5a781b43f2815b.jpg  
 extracting: /content/drive/MyDrive/TRIALDETECTION/unzip/train/01249489-01_jpg.rf.a59115a2ff0c036a5d5a781b43f2815b.xml  
 extracting: /content/drive/MyDrive/TRIALDETECTION/unzip/train/013f014843e29126ff77dbd9556132d3_jpg.rf.aaf9a89321b00a1addf0a0b493c5677e.jpg  
 extracting: /content/drive/MyDrive/TRIALDETECTION/unzip/train/013f014843e29126ff77dbd9556132d3_jpg.rf.aaf9a89321b00a1addf0a0b493c5677e.xml  
 extracting: /content/drive/MyDrive/TRIALDETECTION/unzip/train/01b702e343de4e37410b80fb14618c62_jpg.rf.edfdbe2a52052b079f2128fb46532cb

In [7]:
import os
directory_path = "/content/drive/MyDrive/TRIALDETECTION/unzip/train"
for filename in os.listdir(directory_path):
    print(filename)

01249489-01_jpg.rf.a59115a2ff0c036a5d5a781b43f2815b.jpg
01249489-01_jpg.rf.a59115a2ff0c036a5d5a781b43f2815b.xml
013f014843e29126ff77dbd9556132d3_jpg.rf.aaf9a89321b00a1addf0a0b493c5677e.jpg
013f014843e29126ff77dbd9556132d3_jpg.rf.aaf9a89321b00a1addf0a0b493c5677e.xml
01b702e343de4e37410b80fb14618c62_jpg.rf.edfdbe2a52052b079f2128fb46532cb6.jpg
01b702e343de4e37410b80fb14618c62_jpg.rf.edfdbe2a52052b079f2128fb46532cb6.xml
0377bd4f44b0bb5fb844f82f30295754_jpg.rf.ef203a966c62d976d3d298ac7cd825ab.jpg
0377bd4f44b0bb5fb844f82f30295754_jpg.rf.ef203a966c62d976d3d298ac7cd825ab.xml
045f94b9e6c7682aac426da9e0a797d2_jpg.rf.bca453e89291d22e65bd2747d971bc71.jpg
045f94b9e6c7682aac426da9e0a797d2_jpg.rf.bca453e89291d22e65bd2747d971bc71.xml
05349f4d62cc8cfaa3bd104adf4ba3fd_jpg.rf.aa8532b9eb35178633c90bbd7a8e884e.jpg
05349f4d62cc8cfaa3bd104adf4ba3fd_jpg.rf.aa8532b9eb35178633c90bbd7a8e884e.xml
059296b2105a5642fd365282964bc16c_jpg.rf.85ef8e736c76a64dfc7301eba626b878.jpg
059296b2105a5642fd365282964bc16c_jpg.rf.8

In [4]:
import re
import xml.etree.ElementTree as ET
from PIL import Image


Processing completed.


In [9]:
# Directory containing your XML files and corresponding images
input_directory = '/content/drive/MyDrive/TRIALDETECTION/unzip/train'
xmloutput_directory = '/content/drive/MyDrive/TRIALDETECTION/unzip/train/Annotations'
#imageoutput_directory = '/content/drive/MyDrive/TRIALDETECTION/unzip/train/images'
images_directory = '/content/drive/MyDrive/TRIALDETECTION/unzip/train'

if not os.path.exists(xmloutput_directory):
    os.makedirs(xmloutput_directory)
if not os.path.exists(imageoutput_directory):
    os.makedirs(imageoutput_directory)


In [10]:
def get_new_class_name(old_class_name):
    # Define a mapping of prefixes to new names
    prefix_mapping = {
        "Shoulders_Breadth": "Shoulders",
        "Chest_Breadth": "Chest",
        "Waist_Breadth": "Waist",
        "Legs_Length": "Legs",
        "Legs_Breadth":"LegsBred",
        "ArmR_Length": "ArmRight",
        "ArmL_Length": "ArmLeft",
        "Neck_Depth": "Neck",
        "Hips_Breadth": "Hips",
        "whole": "Whole",
        "UpArmR_Breadth":"UpArmR",
        "UpArmL_Breadth":"UpArmL"
    }
    match = re.match(r"([A-Za-z_]+)_\d+\.\d+", old_class_name)
    if match:
        prefix = match.group(1)
        # Check if the prefix is in our mapping
        if prefix in prefix_mapping:
            # Return the new class name based on the mapping
            return prefix_mapping[prefix]

    # If the old class name doesn't match our pattern or isn't in the mapping,
    # return the old class name unchanged
    return old_class_name


In [11]:
# Process XML files
for filename in os.listdir(input_directory):
    if filename.endswith('.xml'):
        xml_path = os.path.join(input_directory, filename)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Create a new XML tree for the output
        new_root = ET.Element("annotation")
        for elem in root:
            if elem.tag not in ["object"]:  # Copy all elements except 'object'
                new_root.append(elem)

        # Image filename
        image_filename = root.find('filename').text
        image_path = os.path.join(images_directory, image_filename)
        if not os.path.exists(image_path):
            print(f"Image file does not exist for {xml_path}")
            continue

        image = Image.open(image_path)

        '''excluded_classes = ("UpArmR_Breadth_0.0", "UpArmR_Breadth_1.0", "UpArmR_Breadth_2.0", "UpArmR_Breadth_3.0", "UpArmR_Breadth_4.0", "UpArmR_Breadth_5.0", "UpArmR_Breadth_6.0", "UpArmR_Breadth_7.0", "UpArmR_Breadth_8.0", "UpArmR_Breadth_9.0",
                            "UpArmL_Breadth_0.0", "UpArmL_Breadth_1.0", "UpArmL_Breadth_2.0", "UpArmL_Breadth_3.0", "UpArmL_Breadth_4.0", "UpArmL_Breadth_5.0", "UpArmL_Breadth_6.0", "UpArmL_Breadth_7.0", "UpArmL_Breadth_8.0", "UpArmL_Breadth_9.0",
                            "Legs_Breadth_0.0", "Legs_Breadth_1.0", "Legs_Breadth_2.0", "Legs_Breadth_3.0", "Legs_Breadth_4.0", "Legs_Breadth_5.0", "Legs_Breadth_6.0", "Legs_Breadth_7.0", "Legs_Breadth_8.0", "Legs_Breadth_9.0")

        '''

        excluded_classes=()
        # Iterate through objects and process them
        for obj in root.findall('object'):
            class_name = obj.find('name').text
            if class_name in excluded_classes:  # Check if class_name is one of the excluded classes
                continue

            # Add this object to new_root with a sanitized name
            new_obj = ET.Element("object")
            for attr in obj:
                if attr.tag == "name":
                    new_name = ET.SubElement(new_obj, "name")
                    new_name.text = get_new_class_name(class_name)
                else:
                    new_obj.append(attr)
            new_root.append(new_obj)


        # Save the new XML
        new_tree = ET.ElementTree(new_root)
        new_xml_path = os.path.join(xmloutput_directory, f"{filename}")
        new_tree.write(new_xml_path)

print("Processing completed.")

Processing completed.
